### 가상공간 source Seongnam_Scenario_1/bin/activate              
### 파이썬 3.13

In [18]:
import numpy as np 
import pandas as pd 
import geopandas as gpd
import json
import os
import folium 
from requests.adapters import HTTPAdapter 
from urllib3.util.retry import Retry 
from datetime import timedelta
from shapely.geometry import Point 
from ortools.linear_solver import pywraplp 
from tqdm import tqdm 
import pickle
import warnings; warnings.filterwarnings('ignore')

In [19]:
print(os.getcwd())  # 지금 실행되는 작업 디렉토리

/Users/jung-eunjoo/Desktop/은주/연구/AI_agent/Seongnam_Scenario_1/data


In [20]:
boundary_path = "data/seongnam_boundary.geojson"
taxi_path = "Seongnam-si/Seongnam_Taxi/(SNRI) TIMS자료_영업별 자료_QUERRY_4월.xlsx"

In [21]:
taxi_df = pd.read_excel(taxi_path)

 ### 2024-04-18 23:00 ~ 2024-04-19 02:00
 ### 이 시간대 승객들만 정제

In [25]:
import pandas as pd

taxi_df['승차시간'] = pd.to_datetime(taxi_df['승차시간'])

start_time = pd.Timestamp("2024-04-18 23:00:00")
end_time   = pd.Timestamp("2024-04-19 02:00:00")

# 승차시간이 해당 구간에 속하는 경우만 필터링
taxi_df_20240418_2300_20240419_0200 = taxi_df[taxi_df['승차시간'].between(start_time, end_time)]

print(taxi_df.shape)
print(taxi_df[['차량번호','승차시간','하차시간']].head())

(5095, 21)
            차량번호                승차시간                하차시간
59865  경기32바1282 2024-04-18 23:00:12 2024-04-18 23:04:27
59877  경기32바1487 2024-04-18 23:00:14 2024-04-18 23:07:03
59880  경기32바1338 2024-04-18 23:01:37 2024-04-18 23:07:11
59882  경기32바1198 2024-04-18 23:03:24 2024-04-18 23:07:12
59884  경기32바1142 2024-04-18 23:03:09 2024-04-18 23:07:21


In [26]:
taxi_df_20240418_2300_20240419_0200

,구분,차량번호,운송사업자 등록번호,결제일시,승차요금,호출요금,기타요금,할증여부,결제구분(구분불가/현금/카드),승차시간,...,승차Y좌표,승차법정동코드,하차시간,하차X좌표,하차Y좌표,하차법정동코드,승차거리(m),승차전빈차거리(m),승차시간(sec),결제연월일
59865,일반택시,경기32바1282,1298106809,2024-04-18 23:04:33,7500,0,0,할증,현금,2024-04-18 23:00:12,...,37.454429,4113310400,2024-04-18 23:04:27,127.144046,37.447733,4113110100,2499.80,748.31,255,20240418
59877,일반택시,경기32바1487,1298104378,2024-04-18 23:07:09,9400,0,0,할증,카드,2024-04-18 23:00:14,...,37.412327,4113510700,2024-04-18 23:07:03,127.132637,37.381664,4113510500,4050.85,972.46,409,20240418
59880,일반택시,경기32바1338,1298103840,2024-04-18 23:07:12,7200,0,0,할증,카드,2024-04-18 23:01:37,...,37.446423,4113110200,2024-04-18 23:07:11,127.156380,37.451626,4113110400,1884.85,38797.27,334,20240418
59882,일반택시,경기32바1198,1298108447,2024-04-18 23:07:23,7500,0,0,할증,현금,2024-04-18 23:03:24,...,37.433807,4113310100,2024-04-18 23:07:12,127.158401,37.444923,4113310300,2776.79,500.78,228,20240418
59884,일반택시,경기32바1142,6908602687,2024-04-18 23:07:34,8100,0,0,할증,현금,2024-04-18 23:03:09,...,37.430250,4113310100,2024-04-18 23:07:21,127.165958,37.435277,4113310500,3075.51,538.22,252,20240418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908682,개인택시,경기32바6542,3921401939,2024-04-19 01:58:03,17300,0,0,복합할증,현금,2024-04-19 01:40:40,...,37.379269,4113510200,2024-04-19 01:57:55,127.074205,37.329319,4146510500,10490.33,2341.28,1035,20240419
908683,개인택시,경기32바6599,1293769723,2024-04-19 01:58:20,15200,0,0,할증,현금,2024-04-19 01:47:02,...,37.416835,4113510700,2024-04-19 01:58:03,127.113311,37.346983,4113511400,9907.90,2603.55,661,20240419
908684,개인택시,경기32바6605,4312301653,2024-04-19 02:02:49,12100,0,0,할증,카드,2024-04-19 01:51:10,...,37.412369,4113510700,2024-04-19 02:02:44,127.168015,37.451324,4113310300,6558.00,14768.95,694,20240419
908685,개인택시,경기32바7139,1291342164,2024-04-19 02:02:49,8400,0,0,할증,카드,2024-04-19 01:58:36,...,37.409885,4113510700,2024-04-19 02:02:40,127.144676,37.412811,4113510700,2218.50,70611.72,244,20240419


In [27]:
taxi_df_20240418_2300_20240419_0200.columns

Index(['구분', '차량번호', '운송사업자 등록번호', '결제일시', '승차요금', '호출요금', '기타요금', '할증여부',
       '결제구분(구분불가/현금/카드)', '승차시간', '승차X좌표', '승차Y좌표', '승차법정동코드', '하차시간',
       '하차X좌표', '하차Y좌표', '하차법정동코드', '승차거리(m)', '승차전빈차거리(m)', '승차시간(sec)',
       '결제연월일'],
      dtype='object')